<a href="https://colab.research.google.com/github/RG-CyberSleuth/LLM_langchain/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_nvidia_ai_endpoints

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA

In [ ]:
!pip install langchain-community langchain-text-splitters faiss-cpu

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

In [ ]:
docs = loader.load()

In [ ]:
from google.colab import userdata
import os

In [ ]:
os.environ['NVIDIA_API_KEY'] = userdata.get('NVIDIA_API_KEY')


In [ ]:
embeddings = NVIDIAEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

In [ ]:
documents = text_splitter.split_documents(docs)

In [ ]:
documents[0]

Document(page_content='LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'})

In [ ]:
vector = FAISS.from_documents(documents, embeddings)

In [ ]:
retriever = vector.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
model = ChatNVIDIA(model="meta/llama3-70b-instruct")

In [ ]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""

In [ ]:
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)

In [ ]:
hyde_prompt

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:\n\n{question}'))])

In [ ]:
from langchain_core.runnables import chain

In [ ]:
hyde_retriever = hyde_prompt | model | StrOutputParser()

In [ ]:
hyde_retriever

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:\n\n{question}'))])
| ChatNVIDIA(model='meta/llama3-70b-instruct')
| StrOutputParser()

In [ ]:
from langchain_core.runnables import chain

In [ ]:
# Assuming 'retriever' is defined elsewhere and is the intended retriever to use
@chain
def hyde_retriever(question):
    # Use the 'retriever' object directly instead of calling itself
    hypothetical_document = retriever.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
answer_chain = prompt | model | StrOutputParser()

In [ ]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [ ]:
!pip install langchain langchain-community

In [ ]:
!pip install --upgrade langchain langchain-community # Upgrade langchain and langchain-community to the latest version

from langchain.prompts import ChatPromptTemplate # Import ChatPromptTemplate from langchain.prompts
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda

In [ ]:
# Assuming 'retriever' is a previously defined vectorstore retriever
def hyde_retriever(question):
    # Directly use the retriever to fetch relevant documents based on the question
    hypothetical_document = retriever.get_relevant_documents(question)
    return hypothetical_document

In [ ]:
@chain
def final_chain(question):
    documents = hyde_retriever(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [ ]:
for s in final_chain.stream("how can langsmith help with testing"):
    print(s, end="")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


According to the provided context, LangSmith can help with testing in the following ways:

1. A/B testing: LangSmith allows users to create multiple versions of their applications with different identifiers and view their performance side-by-side, making it easier to test changes in prompt, model, or retrieval strategy.
2. Automations: LangSmith's automation feature enables users to perform actions on traces in near real-time, such as automatically scoring traces, sending them to annotation queues, or sending them to datasets.
3. Annotating interesting traces: LangSmith allows users to closely inspect interesting traces and annotate them with respect to different criteria, which helps catch regressions across important evaluation criteria.
4. Adding runs to a dataset: LangSmith enables users to add runs as examples to datasets, which helps refine and improve the application's performance during the beta testing phase.
5. Capturing feedback: LangSmith allows users to gather human feedba